In [1]:
!pip install tensorflow

     |████████████████████████████████| 479.6MB 55.9MB/s eta 0:00:01��███▎    | 409.6MB 58.7MB/s eta 0:00:02     |███████████████████████████████▌| 472.5MB 55.9MB/s eta 0:00:01
     |████████████████████████████████| 24.5MB 45.5MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 46.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 49.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 45.7MB/s eta 0:00:01
     |████████████████████████████████| 1.7MB 47.6MB/s eta 0:00:01
     |████████████████████████████████| 296kB 53.5MB/s eta 0:00:01
     |████████████████████████████████| 901kB 47.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 52.7MB/s eta 0:00:01
     |████████████████████████████████| 5.6MB 45.8MB/s eta 0:00:01
     |████████████████████████████████| 5.3MB 46.9MB/s eta 0:00:01
     |████████████████████████████████| 5.6MB 49.1MB/s eta 0:00:01
     |████████████████████████████████| 450kB 50.5MB/s eta 0:00:01
     |██████████████

     |████████████████████████████████| 184kB 53.6MB/s eta 0:00:01
     |████████████████████████████████| 163kB 51.2MB/s eta 0:00:01
     |████████████████████████████████| 143kB 53.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 46.4MB/s eta 0:00:01
     |████████████████████████████████| 122kB 53.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 51.5MB/s eta 0:00:01
     |████████████████████████████████| 153kB 50.3MB/s eta 0:00:01
ERROR: snowflake-snowpark-python 1.0.0 has requirement snowflake-connector-python<3.0.0,>=2.7.12, but you'll have snowflake-connector-python 3.6.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement absl-py<2,>=0.15, but you'll have absl-py 2.1.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.


In [1]:
from snowflake.snowpark import Session
import configparser

In [3]:
import gc
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
sample_size = None

# Code to establish connection and read data from Snowflake

In [4]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [5]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [6]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [8]:
application_train_sf  = session.table("CRA_APPLICATION_TRAIN_DETAILS")
application_test_sf  = session.table("CRA_APPLICATION_TEST_DETAILS")
bureau_sf  = session.table("CRA_BUREAU_DETAILS")
bureau_balance_sf  = session.table("CRA_BUREAU_BALANCE_DETAILS")
credit_card_balance_sf  = session.table("CRA_CREDIT_CARD_BALANCE_DETAILS")
installments_payments_sf  = session.table("CRA_INSTALLMENTS_PAYMENTS_DETAILS")
previous_application_sf  = session.table("CRA_PREVIOUS_APPLICATION_DETAILS")
POS_CASH_balance_sf  = session.table("CRA_POS_CASH_BALANCE_DETAILS")

# Convert Snowflake data into Pandas dataframes

In [10]:
app_train_df  = application_train_sf.to_pandas()

In [11]:
app_test_df  = application_test_sf.to_pandas()

In [12]:
bureau_df  = bureau_sf.to_pandas()

In [14]:
bureau_balance_df  = bureau_balance_sf.to_pandas()

In [15]:
credit_card_df  = credit_card_balance_sf.to_pandas()

In [16]:
install_df = installments_payments_sf.to_pandas()

In [17]:
prev_app_df = previous_application_sf.to_pandas()

In [18]:
pos_cash_df  = POS_CASH_balance_sf.to_pandas()

In [19]:
print('Data loaded.\nMain application training data set shape = {}'.format(app_train_df.shape))
print('Main application test data set shape = {}'.format(app_test_df.shape))
print('Positive target proportion = {:.2f}'.format(app_train_df['TARGET'].mean()))

Data loaded.
Main application training data set shape = (307511, 124)
Main application test data set shape = (48744, 123)
Positive target proportion = 0.08


In [20]:
app_train_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
app_test_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
bureau_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
bureau_balance_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
credit_card_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
pos_cash_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
prev_app_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
install_df.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)